In [1]:
import requests
import warnings
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook, tnrange

warnings.filterwarnings(action='ignore')
# 경고 메시지를 무시하고 숨기거나 다시보이게 하기('default')

pd.set_option("display.max_rows", None , "display.max_columns", None)
# 데이터 프레임 스크린 뷰 갯수 설정

# requests.get(url)에서
# python httpsconnectionpool error 시 대처 요령 ==================================
# requests 모듈에서 로컬 컴퓨터의 SSL인증(보안)을 확인하는 부분에 문제가 있는 것으로 보인다.
# 1. 관리자 권한으로 VS Code를 실행 후 코드를 동작 시킨다.
# 2. 다음 방법은 보안 연결을 하지 않는 것이다 
# "requests.get(url,verify=False).text"형태로 작성
# ==============================================================================

In [2]:
URL = []      # URL 
category = [] # 카테고리 글

for a in tqdm_notebook(range(100000,100002),desc='대 카테고리'):
    url2 = []     # 중 카테고리   여성의류
    url3 = []     # 소 카테고리   상의
    url4 = []     # 세부 카테고리 티셔츠
    url    = "https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=EC10" + str(a) + "&login=Y&mallId=1"
    source = requests.get(url,verify=False).text
    soup  = BeautifulSoup(source, "html.parser")
    soup2 = soup.select('.srchFilterOptions > li > a')
    for b in soup2:
        url2.append(b['href'])
        
    for c in tqdm_notebook(url2,desc='중 카테고리'):
        source2 = requests.get(c,verify=False).text
        soup3   = BeautifulSoup(source2, "html.parser")
        soup4   = soup3.select('.srchFilterOptions > li > a')
        for d in soup4:
            url3.append(d['href'])
    
    for e in tqdm_notebook(url3,desc='소 카테고리'):
        source3 = requests.get(e,verify=False).text
        soup5   = BeautifulSoup(source3, "html.parser")
        soup6   = soup5.select('.srchFilterOptions > li > a')
        URL = []
        URL.append(e)
        for f in soup6:
            URL.append(f['href'])
        url4.append(URL)   
            
        # 다른 방식
        # for a in soup5.find_all('a', href=True)[4:]:
        #    b.append(a['href'])    

# (코드 수정 전) 카테고리명 한 컬럼에 입력 및 저장 
#    for g in tqdm_notebook(range(len(url5)),desc="세부 카테고리"):
#        source4 = requests.get(url5[g],verify=False).text
#        soup7   = BeautifulSoup(source4, "html.parser")
#        soup8   = soup7.select(".srchBreadCrumb")
#        for h in range(len(soup8)):
#            category.append(soup8[h].text.replace('\n','')
# 
# 
#URL = pd.unique(URL).tolist()
#df = pd.DataFrame([x for x in zip(URL, category)], columns=['url','category'])
#df.to_csv('LOTTEON.csv',encoding='utf-8-sig')

# (코드 수정 후) 각 카테고리명 각 컬럼 할당입력 (+공백제거) ======================
#  DateFrame을 만들기 위한 이중 리스트 생성(2차원 리스트)
    url5 = sum(url4,[])
    url5 = pd.unique(url5).tolist()
    for g in tqdm_notebook(range(len(url5)),desc="세부 카테고리"):
        source4 = requests.get(url5[g],verify=False).text
        soup7   = BeautifulSoup(source4, "html.parser")
        soup8   = soup7.select(".srchBreadCrumb > li")
        line = [] # 안쪽 리스트로 사용할 빈 리스트 생성
        for i in range(1,len(soup8)): # 안쪽 리스트 데이터
            line.append(soup8[i].text.strip()) # 전체 리스트에 안쪽 리스트를 추가
        category.append(line)

    URL = pd.unique(URL).tolist()

# (코드 수정 전) 각 데이터 프레임 생성 후 열 추가방식 ============================
#df        = DataFrame(category,columns=['대','중','소','세부'])
#df['url'] = DataFrame(url5,columns=['url'])
#df        = df[['url','대','중','소','세부']]

# (코드 수정 후) 기준 데이터 프레임 생성 후 원하는 위치에 리스트 열 추가 ============
df = DataFrame(category,columns=['대','중','소','세부'])
df.insert(0,"url",URL,True) 

# 저장
df = df.fillna("NULL")
df['세부'] = df.replace(['NULL'],'') #Null -> 빈칸으로
df.to_csv('LOTTEON.csv',encoding='utf-8-sig')


# ===========================================================================
# 문제) 소 카테고리로 끝난 품목은 세부 카테고리 열에 소 카테고리가 반복해서 입력된다.
# 세부 카테고리(4단계)가 없는 3단계까지의 원피스 url에 들어갈 경우,
# 소 카테고리(3단계)가 반복되어 나타난다. 이로인해 세부 카테고리가 있는 경우임에도 불구하고
# 소 카테고리의 url과 소 카테고리 명까지의 데이터가 중복되어 입력된다.
# (세부 카테고리가 있으면 소 카테고리 url은 제거되어야 한다)

# 패션 > 여성의류 > 상의            (제거)
# 패션 > 여성의류 > 상의 > 티셔츠    (정상)
# 패션 > 여성의류 > 상의 > 니트웨어  (정상)

# 해결 0) 결측값 검사 후 전체를 null 혹은 다른 값으로 수정해준다. ===================
# 데이터 검사 isnull 후 None(Na)값 수정

# na, none null control method
# df['세부'].isnull().sum() # null 갯수
# df['세부'].isnull().all() # 데이터 모두 null인지 검사
# df['세부'].isnull().any() # null이 하나라도 있는지 검사
# df.fillna("NULL")

# Na의 행열에 대한 제거 
# df.dropna() Na값을 하나라도 포함하는 행 삭제
# df.dropna(how='all') 행 전체가 Na인 행만 삭제
# df.dropna(subset=['특정 열']) 특정 열의 Na가 들어있는 행 삭제

# ['소']카테고리의 Na가 들어있는 행은 삭제해야한다.
# 카테고리는 기본 level3까지 있이 떄문에 level2까지의 행은 불필요한 데이터
# df = df.dropna(subset=['소'])

# 그러나 이렇게 하고 해결 1로 들어갔을 때 문제 발생.
# 그래서 처음부터 2단계까지 있는 카테고리를 선별하여 해결1 이후 제거하거나
# df['소'] = df['소'].fillna('no')
# df.fillna({'a':'a','b':'b'}) Na를 각 컬럼별 치환

# 선별 없이 제거하는 것을 생각.
# df.fillna() 전체 치환
df = df.fillna("NULL")

# 해결 1) 세부 카테고리가 없는 행의 소 카테고리 명을 가져오고  a ======================
a = []
b = []
for i in range(len(df)):
    if df['세부'][i] == 'NULL': 
        a.append(df['소'][i])  #세부 카테고리가 없는 소 카테고리
    else:
        a.append('1')
        
# 해결 2) 세부 카테고리가 있는 행의 소 카테고리 명을 가져온다. b (제거목록) ============
    if df['세부'][i] != 'NULL':
        b.append(df['소'][i])  #세부 카테고리가 있는 소 카테고리
    else:
        b.append('NULL')

# 해결 3) 각각 데이터 프레임으로 만든 후, b 데이터의 중복을 제거한다. ==================
# a1b1 = DataFrame([x for x in zip(a, b)],columns=['a','b'])
a1 = DataFrame(a,columns=['a'])                     #세부 카테고리가 없는 소 카테고리
b1 = DataFrame(pd.unique(b).tolist(),columns=['a']) #세부 카테고리가 있는 소 카테고리
b1['b'] = '삭제' #b1['b'] = b를 입력하면 어떤항목이 삭제되는지 보기 쉽다.

# 헤결 4) 중복제거된 b 데이터와 a 데이터의 vlookup을 통해 공통 데이터를 선별한다. =======
delete = a1['a'].map(b1.set_index('a')['b'])

# 해결 5) 공통 데이터(세부 카테고리까지 있음에도 소 카테고리까지만 입력된 행)를 제거한다.
df['delete'] = delete
df.drop(df[df['delete'] == '삭제'].index,inplace=True)
df.drop(['delete'],axis=1,inplace=True) # inplace 결과값 고정
# =============================================================================

# 저장 
df['세부'] = df['세부'].replace(['NULL'],'') #Null -> 빈칸으로
df.to_csv('LOTTEON.csv',encoding='utf-8-sig')

ValueError: Length of values (2) does not match length of index (1084)

In [ ]:
URL = []      # URL 
category = [] # 카테고리 글

for a in tnrange(100000,100002,desc='대 카테고리'):
    url2 = []     # 중 카테고리   여성의류
    #url3         # 소 카테고리   상의
    url4 = []     # 세부 카테고리 티셔츠
    url5 = []
    url    = "https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=EC10" + str(a) + "&login=Y&mallId=1"
    source = requests.get(url,verify=False).text
    soup  = BeautifulSoup(source, "html.parser")
    soup2 = soup.select('.srchFilterOptions > li > a')
    for b in soup2:
        url2 = b['href']
        source2 = requests.get(url2,verify=False).text
        soup3   = BeautifulSoup(source2, "html.parser")
        soup4   = soup3.select('.srchFilterOptions > li > a')
        for c in soup4:
            url3 = c['href']
            source3 = requests.get(url3,verify=False).text
            soup5   = BeautifulSoup(source3, "html.parser")
            soup6   = soup5.select('.srchFilterOptions > li > a') 
            # 세부카테고리가 없으면 소카테고리에서 중 카테고리(이전)이 존재하여 중까지 있는 카테고리가 생성됨
            url4 = []
            url4.append(url3)
            for d in soup6:
                url4.append(d['href'])  

            url5.append(url4)
            
    url5 = sum(url5,[])
    url5 = pd.unique(url5).tolist()
    for g in tnrange(len(url5),desc="세부 카테고리"):
        source4 = requests.get(url5[g],verify=False).text
        soup7   = BeautifulSoup(source4, "html.parser")
        soup8   = soup7.select(".srchBreadCrumb > li")
        line = [] # 안쪽 리스트로 사용할 빈 리스트 생성
        for i in range(1,len(soup8)): # 안쪽 리스트 데이터
            line.append(soup8[i].text.strip()) # 전체 리스트에 안쪽 리스트를 추가
        category.append(line)
        
    URL.append(url5)

In [4]:
URL = []      # URL 
category = [] # 카테고리 글

for a in tqdm_notebook(range(100000,100001),desc='대 카테고리'):
    url2 = []     # 중 카테고리   여성의류
    url3 = []     # 소 카테고리   상의
    url4 = []     # 세부 카테고리 티셔츠
    url    = "https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=EC10" + str(a) + "&login=Y&mallId=1"
    source = requests.get(url,verify=False).text
    soup  = BeautifulSoup(source, "html.parser")
    soup2 = soup.select('.srchFilterOptions > li > a')
    for b in soup2:
        url2.append(b['href'])
        
    for c in tqdm_notebook(url2,desc='중 카테고리'):
        source2 = requests.get(c,verify=False).text
        soup3   = BeautifulSoup(source2, "html.parser")
        soup4   = soup3.select('.srchFilterOptions > li > a')
        for d in soup4:
            url3.append(d['href'])
    
    for e in tqdm_notebook(url3,desc='소 카테고리'):
        source3 = requests.get(e,verify=False).text
        soup5   = BeautifulSoup(source3, "html.parser")
        soup6   = soup5.select('.srchFilterOptions > li > a')
        URL = []
        URL.append(e)
        for f in soup6:
            URL.append(f['href'])
        url4.append(URL)   

    url5 = sum(url4,[])    
    url5 = pd.unique(url5).tolist() 
    for g in tqdm_notebook(url5,desc="세부 카테고리"):
        source4 = requests.get(g,verify=False).text
        soup7   = BeautifulSoup(source4, "html.parser")
        soup8   = soup7.select(".srchBreadCrumb > li")
        line = [] # 안쪽 리스트로 사용할 빈 리스트 생성
        for h in range(1,len(soup8)): # 안쪽 리스트 데이터
            line.append(soup8[h].text.strip()) # 전체 리스트에 안쪽 리스트를 추가
        category.append(line)
        
URL = pd.unique(url).tolist()
# (코드 수정 전) 각 데이터 프레임 생성 후 열 추가방식 ============================
#df        = DataFrame(category,columns=['대','중','소','세부'])
#df['url'] = DataFrame(url5,columns=['url'])
#df        = df[['url','대','중','소','세부']]

# (코드 수정 후) 기준 데이터 프레임 생성 후 원하는 위치에 리스트 열 추가 ============
df = DataFrame(category,columns=['대','중','소','세부'])

KeyboardInterrupt: 

In [5]:
category

[['패션', '여성의류', '원피스'],
 ['패션', '여성의류'],
 ['패션', '여성의류', '상의'],
 ['패션', '여성의류', '하의'],
 ['패션', '여성의류', '정장'],
 ['패션', '여성의류', '여성아우터'],
 ['패션', '여성의류', '시즌/테마'],
 ['패션', '여성의류', '트레이닝복'],
 ['패션', '여성의류', '한복'],
 ['패션', '여성의류', '점프슈트'],
 ['패션', '여성의류', '코디세트'],
 ['패션', '여성의류', '레인코트'],
 ['패션', '여성의류', '상의', '티셔츠'],
 ['패션', '여성의류', '상의', '니트웨어'],
 ['패션', '여성의류', '상의', '가디건'],
 ['패션', '여성의류', '상의', '블라우스'],
 ['패션', '여성의류', '상의', '셔츠'],
 ['패션', '여성의류', '상의', '베스트'],
 ['패션', '여성의류', '상의', '맨투맨'],
 ['패션', '여성의류', '상의', '후드티'],
 ['패션', '여성의류', '상의', '집업'],
 ['패션', '여성의류', '상의', '민소매/탑'],
 ['패션', '여성의류', '상의', '뷔스티에'],
 ['패션', '여성의류', '상의', '볼레로'],
 ['패션', '여성의류', '상의', '숄/망토'],
 ['패션', '여성의류', '상의', '긴팔티셔츠'],
 ['패션', '여성의류', '하의', '바지'],
 ['패션', '여성의류', '하의', '데님'],
 ['패션', '여성의류', '하의', '반바지'],
 ['패션', '여성의류', '하의', '슬랙스'],
 ['패션', '여성의류', '하의', '스커트'],
 ['패션', '여성의류', '하의', '미니스커트'],
 ['패션', '여성의류', '하의', '레깅스'],
 ['패션', '여성의류', '정장', '셔츠'],
 ['패션', '여성의류', '정장', '상의'],
 ['패션', '여성의류', '정장'